# Import Cool Stuff

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai import *
from fastai.vision import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy as sp
import torch
from functools import partial
from sklearn import metrics
from collections import Counter
from fastai.callbacks import *

import PIL
import cv2


# Create Databunch



In [ ]:
path = Path('../input/aptos2019-blindness-detection')

In [ ]:
df = pd.read_csv(path/'train.csv')
df.head()

In [ ]:
src = (
         ImageList.from_df(df, path, folder='train_images', suffix='.png')
        .split_by_rand_pct(0.2, seed=42)
        .label_from_df(cols='diagnosis', label_cls=FloatList)    
        )

In [ ]:
tfms = get_transforms(do_flip=True, flip_vert=True, max_rotate=0.10, max_zoom=1.3, max_warp=0.0, max_lighting=0.2)

In [ ]:
data = (
    src.transform(tfms,size=128)
    .databunch()
    .normalize(imagenet_stats)
)

In [ ]:
# Definition of Quadratic Kappa
from sklearn.metrics import cohen_kappa_score
def quadratic_kappa(y_hat, y):
    return torch.tensor(cohen_kappa_score(torch.round(y_hat), y, weights='quadratic'),device='cuda:0')

# Train your CNN

In [ ]:
!mkdir -p /tmp/.cache/torch/checkpoints
!cp ../input/resnet50/resnet50.pth /tmp/.cache/torch/checkpoints/resnet50-19c8e357.pth

In [ ]:
learn = cnn_learner(data, models.resnet50, metrics=[quadratic_kappa], model_dir = Path('../kaggle/working'),
                   path = Path("."))

In [ ]:
lr = 1e-3
learn.fit_one_cycle(4, lr)

In [ ]:
learn.lr_find()
learn.recorder.plot(suggestion=True)

In [ ]:
learn.unfreeze()
lr = slice(1e-4, 1e-3)
learn.fit_one_cycle(6, max_lr=lr)

In [ ]:
learn.fit_one_cycle(2, max_lr=lr)

## Resize Data and Continue Training

In [ ]:
data = (
    src.transform(tfms,size=224)
    .databunch()
    .normalize(imagenet_stats)
)

In [ ]:
learn.data = data

In [ ]:
lr = 1e-3
learn.fit_one_cycle(4, lr)

In [ ]:
learn.lr_find()
learn.recorder.plot(suggestion=True)

In [ ]:
lr = 1e-5
learn.fit_one_cycle(4, max_lr=lr)

# Create Predictions

In [ ]:
sample_df = pd.read_csv(path/"sample_submission.csv")
sample_df.head()

In [ ]:
learn.data.add_test(ImageList.from_df(sample_df, path, folder='test_images', suffix='.png'))

In [ ]:
preds,y = learn.get_preds(DatasetType.Test)

In [ ]:
int_preds = [int(x) for x in preds]
sample_df.diagnosis = int_preds
sample_df.head()
sample_df.to_csv('submission.csv', index=False)